![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [29]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import time

In [24]:
filename_features = "features.txt"
filename_labels = "activity_labels.txt"

filename_subtrain = "subject_train.txt"
filename_xtrain = "X_train.txt"
filename_ytrain = "y_train.txt"

filename_subtest = "subject_test.txt"
ffilename_xtest = "X_test.txt"
filename_ytest = "y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None,)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None,)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [27]:
%%time
dtc = DecisionTreeClassifier(ccp_alpha=0.001)
dtc

CPU times: total: 0 ns
Wall time: 0 ns


DecisionTreeClassifier(ccp_alpha=0.001)

In [34]:
%%time
# Treinamento do modelo
dtc.fit(X_train, y_train)

CPU times: total: 3.7 s
Wall time: 4.59 s


DecisionTreeClassifier(ccp_alpha=0.001)

In [38]:
%%time
# Previsão e avaliação no conjunto de treinamento
train_preds = dtc.predict(X_train)
train_accuracy = accuracy_score(y_train, train_preds)
train_preds, train_accuracy

CPU times: total: 31.2 ms
Wall time: 32.1 ms


(array([5, 5, 5, ..., 2, 2, 2], dtype=int64), 0.9757889009793254)

In [39]:
%%time
# Previsão e avaliação no conjunto de teste
test_preds = dtc.predict(X_test)
test_accuracy = accuracy_score(y_test, test_preds)
test_preds, test_accuracy

CPU times: total: 15.6 ms
Wall time: 18.1 ms


(array([5, 5, 5, ..., 2, 1, 2], dtype=int64), 0.8791991856124872)

In [41]:
print(f"Acurácia de treinamento: {train_accuracy:.2f}")
print(f"Acurácia de teste: {test_accuracy:.2f}")

Acurácia de treinamento: 0.98
Acurácia de teste: 0.88


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [44]:
%%time
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
X_test_pca, X_train_pca

CPU times: total: 359 ms
Wall time: 128 ms


(array([[-2.68674332],
        [-4.33125548],
        [-4.98536037],
        ...,
        [ 4.55569714],
        [ 3.76462149],
        [ 3.99240165]]),
 array([[-5.5202803 ],
        [-5.53534954],
        [-5.47498801],
        ...,
        [ 5.85750527],
        [ 5.42109482],
        [ 5.49797027]]))

In [46]:
%%time
# Modelo de árvore de decisão
dtc = DecisionTreeClassifier()
dtc

CPU times: total: 0 ns
Wall time: 0 ns


DecisionTreeClassifier()

In [47]:
%%time
# Treinamento do modelo
dtc.fit(X_train_pca, y_train)

CPU times: total: 15.6 ms
Wall time: 23.1 ms


DecisionTreeClassifier()

In [51]:
%%time
# Previsão e avaliação no conjunto de treinamento
train_preds = dtc.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, train_preds)
train_preds, train_accuracy

CPU times: total: 0 ns
Wall time: 5 ms


(array([5, 5, 5, ..., 2, 2, 2], dtype=int64), 1.0)

In [52]:
%%time
# Previsão e avaliação no conjunto de teste
test_preds = dtc.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, test_preds)
test_preds, test_accuracy

CPU times: total: 0 ns
Wall time: 3.99 ms


(array([5, 5, 5, ..., 3, 1, 2], dtype=int64), 0.41024770953512046)

In [53]:
print(f"Acurácia de treinamento: {train_accuracy:.2f}")
print(f"Acurácia de teste: {test_accuracy:.2f}")

Acurácia de treinamento: 1.00
Acurácia de teste: 0.41


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [55]:
# Números de componentes a serem testados
component_numbers = [1, 2, 5, 10, 50]

results = []

In [59]:
for n_components in component_numbers:
    print(f"Testando com {n_components} componentes:")
    start_time = time.time()

    # PCA para reduzir a dimensão
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Modelo de árvore de decisão
    dtc = DecisionTreeClassifier()

    # Treinamento do modelo
    dtc.fit(X_train_pca, y_train)

    # Previsão e avaliação no conjunto de treinamento
    train_preds = dtc.predict(X_train_pca)
    train_accuracy = accuracy_score(y_train, train_preds)

    # Previsão e avaliação no conjunto de teste
    test_preds = dtc.predict(X_test_pca)
    test_accuracy = accuracy_score(y_test, test_preds)

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Armazenando os resultados
    results.append({
        'n_components': n_components,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'time': elapsed_time
    })

    print(f"Acurácia de treinamento: {train_accuracy:.2f}")
    print(f"Acurácia de teste: {test_accuracy:.2f}")
    print(f"Tempo de processamento: {elapsed_time:.4f} segundos")
    print("-" * 40)

Testando com 1 componentes:
Acurácia de treinamento: 1.00
Acurácia de teste: 0.41
Tempo de processamento: 0.1627 segundos
----------------------------------------
Testando com 2 componentes:
Acurácia de treinamento: 1.00
Acurácia de teste: 0.51
Tempo de processamento: 0.1662 segundos
----------------------------------------
Testando com 5 componentes:
Acurácia de treinamento: 1.00
Acurácia de teste: 0.75
Tempo de processamento: 0.1880 segundos
----------------------------------------
Testando com 10 componentes:
Acurácia de treinamento: 1.00
Acurácia de teste: 0.80
Tempo de processamento: 0.2486 segundos
----------------------------------------
Testando com 50 componentes:
Acurácia de treinamento: 1.00
Acurácia de teste: 0.80
Tempo de processamento: 0.7597 segundos
----------------------------------------


In [60]:
results_df = pd.DataFrame(results)
print(results_df)

    n_components  train_accuracy  test_accuracy      time
0              1             1.0       0.410248  0.186808
1              2             1.0       0.510350  0.200401
2              5             1.0       0.752630  0.202343
3             10             1.0       0.790974  0.346205
4             50             1.0       0.800814  1.448436
5              1             1.0       0.410248  0.199577
6              2             1.0       0.519851  0.165786
7              5             1.0       0.743129  0.190835
8             10             1.0       0.791653  0.268477
9             50             1.0       0.807262  0.795903
10             1             1.0       0.410248  0.162704
11             2             1.0       0.513403  0.166210
12             5             1.0       0.748897  0.188008
13            10             1.0       0.796742  0.248632
14            50             1.0       0.802511  0.759747


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

Acurácia: diminui se poucas componentes são usadas, pois informações importantes podem ser perdidas. No entanto, a acurácia  melhora quando o PCA ajuda a reduzir ruídos e redundâncias.

Tempo de Processamento: Geralmente diminui devido à redução no número de variáveis, embora o cálculo do PCA possa adicionar um custo adicional.